<a href="https://colab.research.google.com/github/i-am-akash/Pyspark_Practice/blob/i-am-akash-patch-1/sparkdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spark DataFrame Practice common fns 

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 34.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=12ef31505d658769ff3948bd7a130634069fb76a4d4a83fc504287283c6268bb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [45]:
from pyspark import SparkContext, SparkConf, sql
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
conf = SparkConf().setAppName("rdd_session")
sc = SparkContext.getOrCreate(conf = conf)
spark = SparkSession.builder.appName("df_session").getOrCreate()

#Reading the file using df

---




In [18]:
df = spark.read.options(inferSchema = "True", header = "True").csv("/content/sample_data/california_housing_test.csv")
df.show()
df.printSchema()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -118.36|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|       6.1359|          330000.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  -119.56|   36.51|    

#providing Schema explicilty

In [41]:
schema = StructType([
  StructField("longitude", DoubleType(), True),
  StructField("latitude", DoubleType(), True),
  StructField("housing_median_age", StringType(), True),
  StructField("total_rooms",StringType(), True),
  StructField("total_bedrooms", StringType(), True),
  StructField("population", StringType(), True),
  StructField("households", StringType(), True),
  StructField("median_income", DoubleType(), True),
  StructField("median_house_value", DoubleType(), True)
])

In [48]:
df_schema = spark.read.options(header = "True").schema(schema).csv("/content/sample_data/california_housing_test.csv")
df_schema = df_schema.withColumn("total_rooms", col("total_rooms").cast("Int"))
df_schema.show()
df_schema.printSchema()

+---------+--------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  -122.05|   37.37|         27.000000|       3885|    661.000000|1537.000000| 606.000000|       6.6085|          344700.0|
|   -118.3|   34.26|         43.000000|       1510|    310.000000| 809.000000| 277.000000|        3.599|          176500.0|
|  -117.81|   33.78|         27.000000|       3589|    507.000000|1484.000000| 495.000000|       5.7934|          270500.0|
|  -118.36|   33.82|         28.000000|         67|     15.000000|  49.000000|  11.000000|       6.1359|          330000.0|
|  -119.67|   36.33|         19.000000|       1241|    244.000000| 850.000000| 237.000000|       2.9375|           81700.0|
|  -119.

#Created DF from RDD 